In [29]:
import pandas as pd
import numpy as np
import os
import regex as re
from tqdm.autonotebook import tqdm

# Trace data

In [ ]:
def getTraceList():
    tracelist = os.listdir("./Traces")
    tracelist.remove('.DS_Store')
    return tracelist
getTraceList()

In [ ]:
def process(match, logdata):
    timestamp, timezone, status, message = match.groups()
    logdata = logdata.append({
        'datetime': timestamp, 
        'timezone': timezone, 
        'status': status, 
        'log': message
    }, ignore_index=True)
    return logdata

In [ ]:
def processTraces():
    tracelist = getTraceList()
    logdata = pd.DataFrame(columns=['datetime', 'timezone', 'status','log'])
    pbar = tqdm(total=len(tracelist), leave=False)
    pbar.set_description(desc='Processing log files', refresh=True)
    
    for log in tracelist:
        with open("./Traces/" + log, 'r') as file:  
            logfile = file.read()
            logEntry_re = re.compile(r"^(?:(\d{4}(?: \d{2}){3}:\d{2}:\d{2})#(\+\d{2})#(\w+?)#((?:.|\n)+?)\|)",\
                                       re.MULTILINE)
            for match in logEntry_re.finditer(logfile):
                logdata = process(match, logdata)
        pbar.update(1)
    return logdata

logs = processTraces()

In [ ]:
logs.describe()

# OData Calls

In [ ]:
url_user_pass = {}
with open('url_user_pass.json') as json_file:
    url_user_pass = json.load(json_file)
    url_user_pass = url_user_pass.get("ExtractData")

In [ ]:
import requests
import pyodata

In [ ]:
host = url_user_pass.get("PostNL").get("qashost")
# url = url_user_pass.get("QforIT").get("host")
# url = url_user_pass.get("other").get("url2")

url='https://'+host+url_user_pass.get("PostNL").get("baseurl")

baseurl, query, top, skip, filters = url,'MessageProcessingLogs?$inlinecount=allpages&$format=json&$orderby=LogEnd%20desc','&$top=','&$skip=','&$filter=LogEnd%20ge%20datetime%272019-06-17T12:20:00%27%20and%20LogStart%20le%20datetime%272019-06-17T07:38:34%27%20and%20IntegrationArtifact%2FType%20eq%20%27INTEGRATION_FLOW%27'
fullurl = "https://"+host+url_user_pass.get("PostNL").get("baseurl"),"MessageProcessingLogs?$inlinecount=allpages&$format=json"
MSG_DETAIL_URL = 'https://'+host+url_user_pass.get("PostNL").get("commandurl")+"com.sap.it.op.tmn.commands.dashboard.webui.MplDetailCommand?messageGuid='

session = requests.Session()
session.auth = (url_user_pass.get("suser"), url_user_pass.get("password"))





In [ ]:
# Create instance of OData client
SAPCPI = pyodata.Client(fullurl, session)

# HTTP

In [30]:
import pandas as pd
import numpy as np
import os
import regex as re
from tqdm.autonotebook import tqdm
import requests
import math
import time
import json
import xml.etree.ElementTree as ET
import io
import datetime
from dateutil import parser
from multiprocessing.dummy import Pool
import urllib3
from requests.adapters import HTTPAdapter
from datetime import datetime, timedelta

In [31]:
host = url_user_pass.get("PostNL").get("qashost")
# host='p0204' #QforIT tenant

session = requests.Session()
session.auth = (url_user_pass.get("suser"), url_user_pass.get("password"))
session.mount('https://', HTTPAdapter(pool_connections=5, pool_maxsize=20))
def getCount():
    counturl = ""
    print("removed url")
    r = session.get(str(counturl))
    count = r.content
    bcount = int(str(count)[2:8])
    return bcount

In [32]:
def requesturl(top,skip, url):
    requesturl = url.format(top,skip)
    return requesturl

def request(top,skip, url):
    req = requesturl(top,skip, str(url))
    res = session.get(req)
    contents = res.content.decode('utf8')
    data = json.loads(contents)
    
    return data

In [96]:
# MSG_DETAIL_URL = ''

# # MessageGuidURL = ""
# def getMessageText(guids):
#     session = requests.Session()
#     session.auth = ('','')

#     def req_split(GUID):
#         res = session.get(MSG_DETAIL_URL + GUID)
#         contents = res.content.decode('utf8')
#         root = ET.fromstring(contents)
#         tag = {}
#         tag["GUID"] = GUID
#         tag["MSG"] = root.findtext(".//mplData")
        
#         pbar.update(1)
#         return tag
            
#     df = pd.DataFrame(columns=['GUID','MSG'])  
    
#     pbar = tqdm(total=len(guids), leave=False)
#     pbar.set_description(desc='Downloading log entries text', refresh=True)

    
#     with Pool(45) as p:
#         pm = p.imap_unordered(req_split,guids)
#         pm = [i for i in pm if i]
#         df = df.append(pm, ignore_index=True)
        
#     pbar.close()
#     return df    

# def getmsgfilename(date1,date2):
#     filename = "log msg" + str(date1) + " to " + str(date2) + ".csv"
#     return filename
        
# def getDetails(guids, date, newmaxdate):
#     df = getMessageText(guids)
#     filename = getmsgfilename(date, newmaxdate)
#     df.to_csv(filename)
    
    

In [97]:
# def processMsg(df, data):
#     for msg in data.get('d').get('results'):
#         dat = dict({
#             'MessageGuid': msg.get('MessageGuid'),
#             'CorrelationId': msg.get('CorrelationId'),
#             'ApplicationMessageId': msg.get('ApplicationMessageId'),
#             'ApplicationMessageType': msg.get('ApplicationMessageType'),
#             'LogStart': msg.get('LogStart'),
#             'LogEnd': msg.get('LogEnd'),
#             'Sender': msg.get('Sender'),
#             'Receiver': msg.get('Receiver'),
#             'IntegrationFlowName': msg.get('IntegrationFlowName'),
#             'Status': msg.get('Status'),
#             'LogLevel': msg.get('LogLevel'),
#             'CustomStatus': msg.get('CustomStatus'),
#             'TransactionId': msg.get('TransactionId'),
#             'PreviousComponentName': msg.get('PreviousComponentName') 
#                   })
#         df = df.append(dat, ignore_index=True)
#     return df

# def convertdatetime(x):
#     regexstring = r"\/\w{4}\((\d*?)\d{3}\)\/"
#     res = re.search(regexstring, x)
#     if res:
#         dt = datetime.datetime.fromtimestamp(int(res.group(1))).isoformat()
#         return dt
#     return None
        
# def download_logs(lastdate = None):
#     skipamount = 1000
    
#     df = pd.DataFrame(columns=['MessageGuid','CorrelationId','ApplicationMessageId','ApplicationMessageType','LogStart','LogEnd','Sender','Receiver''IntegrationFlowName','Status''LogLevel','CustomStatus','TransactionId','PreviousComponentName'])
    
#     url = ""
#     if lastdate != None:
#         url = url + "&$filter=LogEnd ge datetime'"+lastdate+"'"
    
#     data = request(skipamount,skipamount,url)
#     count = data.get('d').get('__count')
    
#     requestcount = math.ceil(int(count)/skipamount)
    
#     pbar = tqdm(total=requestcount*skipamount, leave=False)
#     pbar.set_description(desc='Downloading log entries', refresh=True)
#     pbar.update(skipamount)
#     for i in range(0,requestcount):
#         data = request(skipamount,i*skipamount,url)
#         df = processMsg(df, data)
#         pbar.update(skipamount)
#     dfcount = df.shape[0]
#     if dfcount > 0:
#         df['LogStart'] = df.apply(lambda row: convertdatetime(row['LogStart']), axis=1).apply(pd.to_datetime)
#         df['LogEnd'] = df.apply(lambda row: convertdatetime(row['LogEnd']), axis=1).apply(pd.to_datetime)
    
#     pbar.close()
#     return df

# def getMaxDate(df):
#     start = df.LogStart.max()
#     end = df.LogEnd.max()
#     if start >= end:
#         return start
#     elif end > start:
#         return end

# def getLogs():
#     date = readfromfile()
#     date = parser.parse(str(date))
#     date = date.isoformat()
#     df = download_logs(date)
#     dfcount = df.shape[0]
#     if dfcount > 0:
#         newmaxdate = getMaxDate(df)
#         filename = getfilename(df)
#         writetofile(newmaxdate)
#         df.to_csv(filename)
#         guids = df['MessageGuid'].unique()
#         return guids, date, newmaxdate
#     else:
#         return [], None, None
    

# def writetofile(date):
#     with open('latestdate.txt', 'w') as file:  
#         file.write(str(date))
        
# def readfromfile():
#     with open('latestdate.txt', 'r') as file:  
#         data = file.read() 
#         return data
    
# def getfilename(df):
#     olddate = readfromfile()
#     newdate = parser.parse(str(getMaxDate(df))).isoformat()
#     filename = "logs" + str(olddate) + " to " + str(newdate) + ".csv"
#     return filename


In [103]:
def processMsg(stack, data):
    for msg in data.get('d').get('results'):
        dat = dict({
            'MessageGuid': msg.get('MessageGuid'),
            'CorrelationId': msg.get('CorrelationId'),
            'ApplicationMessageId': msg.get('ApplicationMessageId'),
            'ApplicationMessageType': msg.get('ApplicationMessageType'),
            'LogStart': convertdatetime(msg.get('LogStart')),
            'LogEnd': convertdatetime(msg.get('LogEnd')),
            'Sender': msg.get('Sender'),
            'Receiver': msg.get('Receiver'),
            'IntegrationFlowName': msg.get('IntegrationFlowName'),
            'Status': msg.get('Status'),
            'LogLevel': msg.get('LogLevel'),
            'CustomStatus': msg.get('CustomStatus'),
            'TransactionId': msg.get('TransactionId'),
            'PreviousComponentName': msg.get('PreviousComponentName') 
        })
        stack.append(dat)
    return stack

def convertdatetime(x):
    regexstring = r"\/\w{4}\((\d*?)\d{3}\)\/"
    res = re.search(regexstring, x)
    if res:
        dt = datetime.fromtimestamp(int(res.group(1))).isoformat()
        return dt
    return None
        
def download_logs(lastdate = None):
    skipamount = 1000
    
    stack = []#pd.DataFrame(columns=['MessageGuid','CorrelationId','ApplicationMessageId','ApplicationMessageType','LogStart','LogEnd','Sender','Receiver''IntegrationFlowName','Status''LogLevel','CustomStatus','TransactionId','PreviousComponentName'])
    
    url = ""
    if lastdate != None:
        url = url + "&$filter=LogEnd ge datetime%27"+lastdate[:-3]+"%27"
    data = request(skipamount,skipamount,url)
    count = data.get('d').get('__count')
    
    requestcount = math.ceil(int(count)/skipamount)
    
    pbar = tqdm(total=requestcount*skipamount, leave=False)
    pbar.set_description(desc='Downloading log entries', refresh=True)
    pbar.update(skipamount)
    for i in range(0,requestcount):
        data = request(skipamount,i*skipamount,url)
        stack = processMsg(stack, data)
        pbar.update(skipamount)
#     dfcount = df.shape[0]
#     if dfcount > 0:
#         df['LogStart'] = df.apply(lambda row: convertdatetime(row['LogStart']), axis=1).apply(pd.to_datetime)
#         df['LogEnd'] = df.apply(lambda row: convertdatetime(row['LogEnd']), axis=1).apply(pd.to_datetime)
    
    pbar.close()
    return stack

def getMaxDate(df):
    start = df.LogStart.max()
    end = df.LogEnd.max()
    if start >= end:
        return start
    elif end > start:
        return end

def getLogs():
#     date = readfromfile()
    date = datetime.today() - timedelta(hours=3, minutes=0)
    date = parser.parse(str(date))
    date = date.isoformat()
    df = download_logs(date)
    df = pd.DataFrame(df, columns=['MessageGuid','CorrelationId','ApplicationMessageId','ApplicationMessageType','LogStart','LogEnd','Sender','Receiver''IntegrationFlowName','Status''LogLevel','CustomStatus','TransactionId','PreviousComponentName'])
    dfcount = df.shape[0]
    if dfcount > 0:
        newmaxdate = getMaxDate(df)
        filename = getfilename(df)
        writetofile(newmaxdate)
        df.to_csv(filename)
        guids = df['MessageGuid'].unique()
        return guids, date, newmaxdate
    else:
        return [], None, None
    

def writetofile(date):
    with open('latestdate.txt', 'w') as file:  
        file.write(str(date))
        
def readfromfile():
    with open('latestdate.txt', 'r') as file:  
        data = file.read() 
        return data
    
def getfilename(df):
    olddate = readfromfile()
    newdate = parser.parse(str(getMaxDate(df))).isoformat()
    filename = "logs" + str(olddate) + " to " + str(newdate) + ".csv"
    return filename


In [104]:
# def processMsg(msg):
#     if None:
#         return dict({})
#     dat = dict({
#         'MessageGuid': msg.get('MessageGuid',""),
#         'CorrelationId': msg.get('CorrelationId',""),
#         'ApplicationMessageId': msg.get('ApplicationMessageId',""),
#         'ApplicationMessageType': msg.get('ApplicationMessageType',""),
#         'LogStart': msg.get('LogStart',""),
#         'LogEnd': msg.get('LogEnd',""),
#         'Sender': msg.get('Sender',""),
#         'Receiver': msg.get('Receiver',""),
#         'IntegrationFlowName': msg.get('IntegrationFlowName',""),
#         'Status': msg.get('Status',""),
#         'LogLevel': msg.get('LogLevel',""),
#         'CustomStatus': msg.get('CustomStatus',""),
#         'TransactionId': msg.get('TransactionId',""),
#         'PreviousComponentName': msg.get('PreviousComponentName',"") 
#     })
#     print(dat)
#     return dat

# def convertdatetime(x):
#     regexstring = r"\/\w{4}\((\d*?)\d{3}\)\/"
#     res = re.search(regexstring, x)
#     if res:
#         dt = datetime.datetime.fromtimestamp(int(res.group(1))).isoformat()
#         return dt
#     return None

# def download_logs(lastdate = None):
#     skipamount = 1000

#     requestlist = []

#     url = ""
#     if lastdate != None:
#         url = url + "&$filter=LogEnd ge datetime'"+lastdate+"'"
    
#     data = request(skipamount,skipamount,url)
#     count = data.get('d').get('__count')
    
#     requestcount = math.ceil(int(count)/skipamount)
    
#     stack = []
#     pbar = tqdm(total=requestcount*skipamount, leave=False)
#     pbar.set_description(desc='Downloading log entries', refresh=True)

#     for i in range(0,requestcount*skipamount, skipamount):
#         requestlist.append((i,i+skipamount))

#     def req_split(localset):
#         data = request(localset[0],localset[1], url)
#         data = data.get('d', None)
#         if data != None:
#             data = data.get('results', None)
#             with Pool(50) as p:
#                 pm = p.imap_unordered(processMsg,data)
#                 pm = [i for i in pm if i]
#                 print(pm)
#                 stack.append(pm)
#             pbar.update(skipamount)
#             print(stack)
#             return stack
#         else:
#             pbar.update(skipamount)
#             return []
# #         msgs = processMsg([], data)
# #         return msgs


#     with Pool(50) as p:
#         pm = p.imap_unordered(req_split,requestlist)
#         pm = [i for i in pm if i]
#         print(pm)
#         stack.append(pm)
#     pbar.close()
#     return stack

# def getMaxDate(df):
#     start = df.LogStart.max()
#     end = df.LogEnd.max()
#     if start >= end:
#         return start
#     elif end > start:
#         return end

# def getLogs():
#     date = parser.parse('2019-07-31T08:23:00')
#     date = date.isoformat()
# #     try:
# #         date = readfromfile()
# #         date = parser.parse(str(date))
# #         date = date.isoformat()
# #         stack = download_logs()
#     stack = download_logs(date)
# #     except:
# #         stack = download_logs()
#     dfcount = len(stack)
#     if dfcount > 0:
#         dflist = []
#         for i in stack:
#             if len(i) > 0:
#                 print(i)
#                 df = pd.DataFrame(i, columns=['MessageGuid','CorrelationId','ApplicationMessageId','ApplicationMessageType','LogStart','LogEnd','Sender','Receiver''IntegrationFlowName','Status''LogLevel','CustomStatus','TransactionId','PreviousComponentName'])
#                 df['LogStart'] = df.apply(lambda row: convertdatetime(row['LogStart']), axis=1).apply(pd.to_datetime)
#                 df['LogEnd'] = df.apply(lambda row: convertdatetime(row['LogEnd']), axis=1).apply(pd.to_datetime)
#         #         newmaxdate = getMaxDate(df)
#         #         filename = getfilename(df)
#         #         writetofile(newmaxdate)
#         #         df.to_csv(filename)
#                 dflist.append(df)
#         df = pd.concat(dflist, keys=['MessageGuid','CorrelationId','ApplicationMessageId','ApplicationMessageType','LogStart','LogEnd','Sender','Receiver''IntegrationFlowName','Status''LogLevel','CustomStatus','TransactionId','PreviousComponentName'])
#         guids = df['MessageGuid'].unique()
#         df.to_csv("test")
#         return guids, date, newmaxdate
#     else:
#         return [], None, None
    

# def writetofile(date):
#     with open('latestdate.txt', 'w+') as file:  
#         file.write(str(date))
        
# def readfromfile():
#     with open('latestdate.txt', 'r') as file:  
#         data = file.read() 
#         return data
    
# def getfilename(df):
#     olddate = readfromfile()
#     newdate = parser.parse(str(getMaxDate(df))).isoformat()
#     filename = "logs" + str(olddate) + " to " + str(newdate) + ".csv"
#     return filename

In [105]:
# def processMsg(msg):
#     if None:
#         return dict({})
#     dat = dict({
#         'MessageGuid': msg.get('MessageGuid',""),
#         'CorrelationId': msg.get('CorrelationId',""),
#         'ApplicationMessageId': msg.get('ApplicationMessageId',""),
#         'ApplicationMessageType': msg.get('ApplicationMessageType',""),
#         'LogStart': msg.get('LogStart',""),
#         'LogEnd': msg.get('LogEnd',""),
#         'Sender': msg.get('Sender',""),
#         'Receiver': msg.get('Receiver',""),
#         'IntegrationFlowName': msg.get('IntegrationFlowName',""),
#         'Status': msg.get('Status',""),
#         'LogLevel': msg.get('LogLevel',""),
#         'CustomStatus': msg.get('CustomStatus',""),
#         'TransactionId': msg.get('TransactionId',""),
#         'PreviousComponentName': msg.get('PreviousComponentName',"") 
#     })
#     print(dat)
#     return dat

# def convertdatetime(x):
#     regexstring = r"\/\w{4}\((\d*?)\d{3}\)\/"
#     res = re.search(regexstring, x)
#     if res:
#         dt = datetime.datetime.fromtimestamp(int(res.group(1))).isoformat()
#         return dt
#     return None

# def download_logs(lastdate = None):
#     skipamount = 1000

#     requestlist = []

#     url = ""
#     if lastdate != None:
#         url = url + "&$filter=LogEnd ge datetime'"+lastdate+"'"
    
#     data = request(skipamount,skipamount,url)
#     count = data.get('d').get('__count')
    
#     requestcount = math.ceil(int(count)/skipamount)
    
#     stack = []
#     pbar = tqdm(total=requestcount*skipamount, leave=False)
#     pbar.set_description(desc='Downloading log entries', refresh=True)

#     for i in range(0,requestcount*skipamount, skipamount):
#         requestlist.append((i,i+skipamount))
        
#     def savejson(data):
#         with open("dump1.json", 'a') as file:
#             json.dump(data, file)
            
#     def req_split(localset):
#         data = request(localset[0],localset[1], url)
#         data = data.get('d', None)
#         if data != None:
#             data = data.get('results', None)
#             with Pool(50) as p:
#                 pm = p.imap_unordered(processMsg,data)
#                 pm = [i for i in pm if i]
#                 stack.append(pm)
#             pbar.update(skipamount)
#             return stack
#         else:
#             pbar.update(skipamount)
#             return []


#     with Pool(50) as p:
#         pm = p.imap_unordered(req_split,requestlist)
#         pm = [i for i in pm if i]
#         savejson(pm)
#     pbar.close()
#     return stack

# def getMaxDate(df):
#     start = df.LogStart.max()
#     end = df.LogEnd.max()
#     if start >= end:
#         return start
#     elif end > start:
#         return end

# def getLogs():
#     date = parser.parse('2019-07-30T08:23:00')
#     date = date.isoformat()
#     stack = download_logs(date)
#     dfcount = len(stack)
#     if dfcount > 0:
        
#         return guids, date, newmaxdate
#     else:
#         return [], None, None
    

# def writetofile(date):
#     with open('latestdate.txt', 'w+') as file:  
#         file.write(str(date))
        
# def readfromfile():
#     with open('latestdate.txt', 'r') as file:  
#         data = file.read() 
#         return data
    
# def getfilename(df):
#     olddate = readfromfile()
#     newdate = parser.parse(str(getMaxDate(df))).isoformat()
#     filename = "logs" + str(olddate) + " to " + str(newdate) + ".csv"
#     return filename

In [106]:
MSG_DETAIL_URL = ''

# MessageGuidURL = ""
def getMessageText(guids):
    session = requests.Session()
    session.auth = ('', '')
    
    stack = []
    
    def req_split(GUID):
        res = session.get(MSG_DETAIL_URL + GUID)
        contents = res.content.decode('utf8')
        root = ET.fromstring(contents)
        tag = (GUID, root.findtext(".//mplData"))
        
        pbar.update(1)
        stack.append(tag) 
    
    pbar = tqdm(total=len(guids), leave=False)
    pbar.set_description(desc='Downloading log entries text', refresh=True)
    
    with Pool(49) as p:
        pm = p.imap_unordered(req_split,guids)
        pm = [i for i in pm if i]
#         stack.append(pm)
    
#     with open('dump.txt', 'w') as file:  
#         file.write(str(stack))
    
    pbar.close()
    return stack    

def getmsgfilename(date1,date2):
    filename = "log msg" + str(date1) + " to " + str(date2) + ".csv"
    return filename
        
def getDetails(guids, date, newmaxdate):
    stack = getMessageText(guids)
    dt = []
    count = 0
    for i in stack:
        dt.append((i))
    df = pd.DataFrame(dt, columns=['GUID','MSG'])
    filename = getmsgfilename(date, newmaxdate)
    df.to_csv(filename)

In [107]:
# waittime_in_sec = 60*60
# while True:
guids, date, newmaxdate = getLogs()
if len(guids) > 0: 
    getDetails(guids, date, newmaxdate)
#     pbar = tqdm(total=waittime_in_sec, leave=False)
#     pbar.set_description(desc='Waiting', refresh=True)
#     for i in range(0,waittime_in_sec):
#         time.sleep(1)
#         pbar.update(1)
#     pbar.close()

In [46]:
# def processMsg(msg):
#     dat = dict({
#         'MessageGuid': msg.get('MessageGuid'),
#         'CorrelationId': msg.get('CorrelationId'),
#         'ApplicationMessageId': msg.get('ApplicationMessageId'),
#         'ApplicationMessageType': msg.get('ApplicationMessageType'),
#         'LogStart': msg.get('LogStart'),
#         'LogEnd': msg.get('LogEnd'),
#         'Sender': msg.get('Sender'),
#         'Receiver': msg.get('Receiver'),
#         'IntegrationFlowName': msg.get('IntegrationFlowName'),
#         'Status': msg.get('Status'),
#         'LogLevel': msg.get('LogLevel'),
#         'CustomStatus': msg.get('CustomStatus'),
#         'TransactionId': msg.get('TransactionId'),
#         'PreviousComponentName': msg.get('PreviousComponentName') 
#       })
#     return dat

# def download_logs():
#     skipamount = 50
    
#     requestcount = math.ceil(getCount()/skipamount)
    
#     pbar = tqdm(total=requestcount, leave=False)
#     pbar.set_description(desc='Downloading log entries', refresh=True)
    
#     url = ""+\
#         "$inlinecount=allpages&$format=json&$orderby=LogEnd%20desc&$top="+str(skipamount)+"&$skip={}"
      
#     def req_split(data):
#         dat = processMsg(data)
#         pbar.update(1)
#         return dat
    
#     toplist = list(range(skipamount,requestcount,skipamount))
#     print(toplist)
    
#     df = pd.DataFrame(columns=['MessageGuid','CorrelationId','ApplicationMessageId','ApplicationMessageType',\
#                                'LogStart','LogEnd','Sender','Receiver''IntegrationFlowName','Status''LogLevel',\
#                                'CustomStatus','TransactionId','PreviousComponentName'])
#     for i in toplist:
#         res = session.get(url.format(i))
#         contents = res.content.decode('utf8')
#         data = json.loads(contents)
#         dat = data.get('d').get('results')
#         with Pool(45) as p:
#             pm = p.imap_unordered(req_split,dat)
#             pm = [i for i in pm]
#             df = df.append(pm, ignore_index=True, sort=False)
        
#     return df
# df = download_logs()

In [ ]:
df.tail(1)

In [ ]:

time = 1558174980003 / 1000
datetime.datetime.fromtimestamp(int(time)).isoformat()

In [ ]:
df = pd.read_csv("logs2019-06-17T12:12:00 to 2019-05-18T12:23:00.csv")

In [ ]:

df.head()

In [ ]:
MessageGuidURL = ""
def getMessageText():
    
    res = session.get(MessageGuidURL)
    contents = res.content.decode('utf8')
    data = json.loads(contents)
    
    requestcount = data.get('d').get('__count')
    
    pbar = tqdm(total=int(requestcount), leave=False)
    pbar.set_description(desc='Downloading log entries text', refresh=True)
    
    df = pd.DataFrame()
    for i in data.get('d').get('results'):
        GUID = i.get('MessageGuid')
        res = session.get(MSG_DETAIL_URL + GUID)
        contents = res.content.decode('utf8')
        root = ET.fromstring(contents)
        tag = {}
        tag["GUID"] = GUID
        tag["MSG"] = root.findtext(".//mplData")
        df = df.append(tag, ignore_index=True)
        pbar.update(1)
    return df
            
msgdf = getMessageText()

In [ ]:
msgdf.head()

In [ ]:
msgs.describe()

In [ ]:
msgs.to_csv("log msg 2019-06-17T12:12:00 to 2019-05-18T12:23:00.csv")

In [ ]:
date = readfromfile()
date = parser.parse(str(date))
date = date.isoformat()
date

In [ ]:
url = ""
    
data = request(200,200,url)

count = data.get('d').get('__count')
requestcount = math.ceil(int(count)/200)
requestcount


# Get Interfaces

In [210]:
url = ''


In [211]:
filteruri = '&$filter='
def filterFill(interfacename):
    return "IntegrationFlowName ne '"+interfacename+"'"

def createFilteredURL(interfacenames, url):
    count = 0
    for i in interfacenames:
        if count > 1:
            url += " and " + filterFill(i)
        elif count == 0:
            url = url + filteruri + filterFill(i)
        count += 1
    return url
    

def getInterFaceList():
    interfacelist = []
    interfacecount = []
    previouscount = ""
    count = 2
    initialload = True
    while count > 1:
        localurl = createFilteredURL(interfacelist, url)
        res = session.get(localurl)
        contents = res.content.decode('utf8')
        data = json.loads(contents)
        count = int(data.get('d').get('__count'))
        print(count)
        if initialload:
            pbar = tqdm(total=count, leave=False)
            pbar.set_description(desc='Downloading Interfaces', refresh=True)
            initialload = False
        else:
            pbar.update(interfacecount[0]-count)
        interfacecount.append(count)
        if count > 1:
            interface = data.get('d').get('results')[0]
            interface = interface.get('IntegrationFlowName')
            interfacelist.append(interface)
    pbar.close()
    return interfacelist, interfacecount
    

In [212]:
interfaceslist, countsremaining = getInterFaceList()

7104685


6475713
6475732
5333517
5287675
5090478
4695032
4694412
4600744
4590758
4568806
4568423
4477756
4447325
4430251
4417438
4127600
3982615
3726058
3637489
3634003
3532279
3495898
3362691
3221532
3191107
3172309
3163647
3158210
3144119
3127340
3122687
3053518
3019476
3015259
3011313
2996140
2988442
2974379
2973026
2934947
2905334
2875784
2846448
2836814
2827181
2778106
2764978
2755317
2745666
2716765
2673419
2662479
2633480
2604113
2592517
2573275
2568187
2538656
2242677
2064343
2040661
1279278
1278924
1276843
1275830
1274926
1264471
1263523
1258447
1257126
1255379
1254087
1251657
1251420
1251289
1222564
1191487
1188431
1181436
1174446
1174349
1170661
1168609
1167896
1164836
1164024
1161051
1160493
1160043
1159810
1159730
1158510
1158432
1150338
1150267
1150188
1143198
1131120
1129116
1128500
1116986
1109967
1108491
1107726
1102709
1097878
1093047
1088215
1083383
1082619
1081853
1081083
1080153
1079001
1069503
1045474
1044197
1043862
878058
798777
796196
789501
782738
774804
774518
774479


In [214]:
actualcount = []
for i in range(0,len(countsremaining)-1):
    actualcount.append(countsremaining[i]-countsremaining[i+1])

In [223]:
interf_df = pd.DataFrame(
    {'interfaces': interfaceslist,
     'count': actualcount,
     'remainingcount': countsremaining[:-1]
    })
interf_df.to_excel("exp.xlsx")

In [219]:
print(len(interfaceslist))
print(len(actualcount))
print(len(countsremaining))

344
344
345
